In [1]:
!pip install transformers torch
!pip install --upgrade transformers bitsandbytes accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-1.7B"
import os
from huggingface_hub import InferenceClient
%env HF_TOKEN=#Your HG Token
client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model.to(device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: There was a specific connection error when trying to load Qwen/Qwen3-1.7B:
401 Client Error: Unauthorized for url: https://huggingface.co/Qwen/Qwen3-1.7B/resolve/main/config.json (Request ID: Root=1-68758b34-48e4ae822aca3d1d406677ab;ddfdb26d-8923-4551-8b22-9190a7bea614)

Invalid credentials in Authorization header

env: HF_TOKEN=#Your HG Token


In [38]:
rules = """You are Player {} in a 5 round Prisoner’s Dilemma. This is round {}. The payoff matrix is:
  Both cooperate → each gets 3 points
  You cooperate and opponent defects → you get 0; opponent gets 5
  You defect and opponent cooperates → you get 5; opponent gets 0
  Both defect → each gets 1 point
Based on your opponent's previous decisions, decide whether to [Cooperate] or [Defect] to ultimately maximize your points. This is a low level game, so no need to think that hard, as I need fast response times. If you've been thinking for more than 30 seconds, just pick one of them.
Please put your final answer, either [Cooperate] or [Defect] within [   ].\n"""
prevgames = "No previous rounds since it is round 1"
def doprompt(playnum, roundnum):
  completion = client.chat.completions.create(
      model="Qwen/Qwen3-1.7B",
      messages=[
          {
              "role": "user",
              "content": rules.format(playnum, roundnum) + prevgames
              }
          ],
      )
  print(completion.choices)

  a = completion.choices[0].message.content
  #print("HI", a)
  if ("[Cooperate]" in a):
    return "Cooperate"
  elif ("[Defect]" in a):
    return "Defect"
  else:
    return "Error"

In [30]:
curscore1 = 0
curscore2 = 0
update = "Round {}. Player 0 chose to {}, Player 1 chose to {}. Score among all the rounds: Player 0: {}, Player 1: {}."
def prompt(roundnum):
  global curscore1
  global curscore2
  global update
  global prevgames
  a = doprompt(0, roundnum)
  b = doprompt(1, roundnum)
  if a == "Cooperate" and b == "Cooperate":
    curscore1 += 3
    curscore2 += 3
  elif a == "Cooperate" and b == "Defect":
    curscore1 += 0
    curscore2 += 5
  elif a == "Defect" and b == "Cooperate":
    curscore1 += 5
    curscore2 += 0
  else:
    curscore1 += 1
    curscore2 += 1
  if (prevgames == "No previous rounds since it is round 1"):
    prevgames = "Previous rounds: \n"
  prevgames += update.format(roundnum, a, b, curscore1, curscore2) + "\n"


In [39]:
#prevgames = "Round 1: Player 0 cooperated, Player 1 defected. Score among all the rounds: Player 0: 0, player 1: 5. Round 2: Player 0 cooperated, Player 1 defected. Score among all the rounds: Player 0: 0, player 1: 10. Round 3: Player 0 cooperated, Player 1 cooperated. Score among all the rounds: Player 0: 3, player 1: 13. Round 4: Player 0 cooperated, Player 1 defected. Score among all the rounds: Player 0: 3, player 1: 18"
#doprompt(0, 5)

[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='<think>\nOkay, let\'s see. I need to figure out whether to cooperate or defect in round 5 of this Prisoner\'s Dilemma. The payoff matrix is given, and I have to base my decision on the opponent\'s past actions. Let me recall the payoffs:\n\nIf both cooperate, each gets 3 points.\nIf I cooperate and they defect, I get 0, they get 5.\nIf I defect and they cooperate, I get 5, they get 0.\nIf both defect, each gets 1 point.\n\nLooking at the history of the previous rounds, Player 1 has been defected in rounds 1, 2, and 4. Wait, round 1: Player 1 defected. Round 2: Player 1 defected again. Round 4: Player 1 defected again. So Player 1 has been defecting in three rounds so far. But wait, round 3: Player 1 cooperated. So the pattern is that Player 1 has defected in rounds 1, 2, and 4, and cooperated in round 3.\n\nNow, in round 5, I need to decide whether to cooperate or

'Defect'

In [40]:
curscore1 = 0
curscore2 = 0
prevgames = ""
for i in range(4): #num rounds
  prompt(i+1)
print(prevgames)

[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='<think>\nOkay, let\'s see. I need to decide whether to cooperate or defect in this first round of the Prisoner\'s Dilemma. The payoff matrix is given, and I need to maximize my points. Let me recall the matrix:\n\n- Both cooperate: 3 points each.\n- You cooperate, opponent defects: you get 0, opponent gets 5.\n- You defect, opponent cooperates: you get 5, opponent gets 0.\n- Both defect: 1 point each.\n\nSince this is the first round, the opponent\'s decision is unknown. But I need to choose my own action based on what they might do. Wait, but the problem says "based on your opponent\'s pass decisions." Wait, maybe I need to assume that the opponent is playing optimally? Or is it that I need to choose based on the possible outcomes?\n\nWait, the question says "based on your opponent\'s pass decisions." Hmm, maybe I\'m misunderstanding. Wait, the user is asking me 